# Recurrent Neural Networks

calcular o preço das ações do google

In [7]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Importing the training set
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

In [9]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [10]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(40, 1258):
    X_train.append(training_set_scaled[i-40:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train

array([[[0.08581368],
        [0.09701243],
        [0.09433366],
        ...,
        [0.04286033],
        [0.04602277],
        [0.05398467]],

       [[0.09701243],
        [0.09433366],
        [0.09156187],
        ...,
        [0.04602277],
        [0.05398467],
        [0.05738894]],

       [[0.09433366],
        [0.09156187],
        [0.07984225],
        ...,
        [0.05398467],
        [0.05738894],
        [0.05714711]],

       ...,

       [[0.96054394],
        [0.9371419 ],
        [0.92841729],
        ...,
        [0.95475854],
        [0.95204256],
        [0.95163331]],

       [[0.9371419 ],
        [0.92841729],
        [0.90804747],
        ...,
        [0.95204256],
        [0.95163331],
        [0.95725128]],

       [[0.92841729],
        [0.90804747],
        [0.8771858 ],
        ...,
        [0.95163331],
        [0.95725128],
        [0.93796041]]])

In [11]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first GRU layer and some Dropout regularisation
regressor.add(GRU(units = 30, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.1))

# Adding a second GRU layer and some Dropout regularisation
# regressor.add(GRU(units = 60, return_sequences = True))
# regressor.add(Dropout(0.1))

# Adding a third GRU layer and some Dropout regularisation
regressor.add(GRU(units = 90, return_sequences = True))
# regressor.add(Dropout(0.1))

# Adding a fourth GRU layer and some Dropout regularisation
regressor.add(GRU(units = 60))
regressor.add(Dropout(0.1))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [12]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 10, batch_size = 32)

Epoch 1/10
1218/1218 [==============================] - 13s 11ms/step - loss: 0.0326 3s - l
Epoch 2/10
1218/1218 [==============================] - 10s 8ms/step - loss: 0.0025
Epoch 3/10
1218/1218 [==============================] - 8s 7ms/step - loss: 0.0021
Epoch 4/10
1218/1218 [==============================] - 8s 7ms/step - loss: 0.0020
Epoch 5/10
1218/1218 [==============================] - 9s 8ms/step - loss: 0.0019
Epoch 6/10
1218/1218 [==============================] - 13s 11ms/step - loss: 0.0018
Epoch 7/10
1218/1218 [==============================] - 10s 9ms/step - loss: 0.0019
Epoch 8/10
1218/1218 [==============================] - 14s 11ms/step - loss: 0.0017
Epoch 9/10
1218/1218 [==============================] - 11s 9ms/step - loss: 0.0016
Epoch 10/10
1218/1218 [==============================] - 8s 7ms/step - loss: 0.0018


In [13]:
# Part 3 - Making the predictions and visualising the results

# Getting the real stock price of 2017
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 40:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(40, 60):
    X_test.append(inputs[i-40:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

NameError: name 'dataset_test' is not defined

In [14]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

NameError: name 'real_stock_price' is not defined

## FIM